In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from sympy.solvers import solve
from sympy import symbols, Eq
import elegansfunc as elegans


dataset=pd.read_excel('./dataset.xlsx')

by_15C=dataset[dataset['temperature']==15]
by_20C=dataset[dataset['temperature']==20]
by_25C=dataset[dataset['temperature']==25]

In [2]:
neuron=elegans.define_model_interactions('nsm',TA2TN=1,TA2S=1,TA2DA=1,DA2TA=-1,DA2S=-1,DA2TN=-1,TN2S=1)

In [3]:
neuron

,TAnsm,TNnsm,DAnsm,Snsm
TAnsm,0,1,1,1
TNnsm,0,0,0,1
DAnsm,-1,-1,0,-1


In [4]:
neuron_temp=neuron.copy()

In [5]:
S='S'+'nsm'
neuron_temp.columns=neuron_temp.columns+('*ADF','*NSM','*ASI','')
neuron_temp.index=neuron_temp.index+('*ADF','*NSM','*ASI')

In [6]:
neuron_temp

,TAnsm*ADF,TNnsm*NSM,DAnsm*ASI,Snsm
TAnsm*ADF,0,1,1,1
TNnsm*NSM,0,0,0,1
DAnsm*ASI,-1,-1,0,-1


In [112]:
interactions_name=list(neuron_temp.index)
neuron_label='nsm'

###activatory_interactions
direct_activatory_interactions_to_cell=[interactions_name[i] for i in np.where(neuron_temp['S{}'.format(neuron_label)]==1)[0]] #anything that directly activate the cell

indirect_activation_to_activatory_interactions_to_cell=[interactions_name[i] for n in direct_activatory_interactions_to_cell for i in np.where(neuron_temp[n]==1)[0]]#anything that activate the direct activation of the cell.

indirect_inhibition_to_activatory_interactions_to_cell=dict()
for n in direct_activatory_interactions_to_cell:
    indirect_inhibition_to_activatory_interactions_to_cell[n]=[interactions_name[i] for i in np.where(neuron_temp[n]==-1)[0]]
for i in indirect_inhibition_to_activatory_interactions_to_cell:
    if len(indirect_inhibition_to_activatory_interactions_to_cell[i])==0:
        indirect_inhibition_to_activatory_interactions_to_cell[i]='1'
    else:
        indirect_inhibition_to_activatory_interactions_to_cell[i]='('+'+'.join(['1']+indirect_inhibition_to_activatory_interactions_to_cell[i])+')'




###inhibitory_interactions these will be all under the S
direct_inhibitory_interactions_to_cell=[interactions_name[i] for i in np.where(neuron_temp['S{}'.format(neuron_label)]==-1)[0]]

indirect_activation_to_inhibitory_interactions_to_cell=[interactions_name[i] for n in direct_inhibitory_interactions_to_cell for i in np.where(neuron_temp[n]==1)[0]]

indirect_inhibition_to_inhibitory_interactions_to_cell=dict()
for n in direct_inhibitory_interactions_to_cell:
    indirect_inhibition_to_inhibitory_interactions_to_cell[n]=[interactions_name[i] for i in np.where(neuron_temp[n]==-1)[0]]
for i in indirect_inhibition_to_inhibitory_interactions_to_cell:
    if len(indirect_inhibition_to_inhibitory_interactions_to_cell[i])==0:
        indirect_inhibition_to_inhibitory_interactions_to_cell[i]='1'
    else:
        indirect_inhibition_to_inhibitory_interactions_to_cell[i]='('+'+'.join(['1']+indirect_inhibition_to_inhibitory_interactions_to_cell[i])+')'





###build term by term:
#term Snsm:

denominator_of_S=[]
for i in list(indirect_inhibition_to_inhibitory_interactions_to_cell.keys()):
    term='/'.join([i,list(indirect_inhibition_to_inhibitory_interactions_to_cell.values())[0]])
    denominator_of_S.append(term)
for i in indirect_activation_to_inhibitory_interactions_to_cell:
    denominator_of_S.append(i)
if denominator_of_S==[]:
    denominator_of_S=['1']
else:
    denominator_of_S=['('+'+'.join(['1']+denominator_of_S)+')']
S_term=['/'.join(['S{}'.format(neuron_label)]+denominator_of_S)]

##term for direct activation of S:
direct_activation_term=[]
for i in list(indirect_inhibition_to_activatory_interactions_to_cell.keys()):
    term='/'.join([i,list(indirect_inhibition_to_activatory_interactions_to_cell.values())[0]])
    direct_activation_term.append(term)
for i in indirect_activation_to_activatory_interactions_to_cell:
    direct_activation_term.append(i)
direct_activation_term=['+'.join(direct_activation_term)]


equation='+'.join(S_term+direct_activation_term)+'-alpha*{}'.format(neuron_label.upper())





In [113]:

equation

'Snsm/(1+DAnsm*ASI/1+TAnsm*ADF)+TAnsm*ADF/(1+DAnsm*ASI)+TNnsm*NSM/(1+DAnsm*ASI)+TAnsm*ADF-alpha*NSM'

In [103]:
degradation_term

['alpha*NSM']

In [111]:
direct_activation_term=[]

In [97]:
direct_activation_term

['TAnsm*ADF/(1+DAnsm*ASI)+TNnsm*NSM/(1+DAnsm*ASI)+TAnsm*ADF']

In [90]:
denominator_of_direct_activation

['TAnsm*ADF/(1+DAnsm*ASI)', 'TNnsm*NSM/(1+DAnsm*ASI)', 'TAnsm*ADF']

In [86]:
indirect_inhibition_to_activatory_interactions_to_cell

{'TAnsm*ADF': '(1+DAnsm*ASI)', 'TNnsm*NSM': '(1+DAnsm*ASI)'}

In [75]:
len('+'.join([]))

0

In [42]:

list(indirect_inhibition_to_inhibitory_interactions_to_cell.keys())

['DAnsm*ASI']

In [61]:

test=indirect_inhibition_to_inhibitory_interactions_to_cell

In [62]:

test.keys()

dict_keys(['DAnsm*ASI'])

In [64]:
'/'.join([list(test.keys())[0],list(test.values())[0]])

'DAnsm*ASI/1'

In [37]:
indirect_inhibition_to_inhibitory_interactions_to_cell

{'DAnsm*ASI': ['1']}

In [35]:
indirect_inhibition_to_activatory_interactions_to_cell

{'TAnsm*ADF': '(1+DAnsm*ASI)', 'TNnsm*NSM': '(1+DAnsm*ASI)'}

In [18]:
indirect_inhibition_to_inhibitory_interactions_to_cell

{'DAnsm*ASI': ['(1+', ')']}

In [23]:
test={'a':['b','c','d']}

In [32]:
'('+'+'.join(test['a']+['1'])+')'

'(b+c+d+1)'

In [57]:
inhibitory_to_cell['Snsm']

['TNnsm*NSM', 'DAnsm*ASI']

Constraint: It should interact with connection to the cell.
Only 1 connection to either the cell or to the self-regulation of the cell


In [54]:
inhibitory_to_cell

{'Snsm': ['TNnsm*NSM', 'DAnsm*ASI'], 'TAnsm*ADF': ['DAnsm*ASI']}

In [45]:
combined_activatory_to_cell

['Snsm', 'TAnsm*ADF']

In [49]:
del temp

In [44]:
activatory_to_connections

[]